In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_Y = pd.read_csv("train_2016_v2.csv", index_col = 'parcelid', parse_dates=['transactiondate'])

In [3]:
allX = pd.read_csv("properties_2016.csv", index_col = 'parcelid')

C:\Users\Ivan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Ivan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
allX['hashottuborspa'] = allX["hashottuborspa"].fillna(False).astype(bool).astype("int64")
allX["propertycountylandusecode"] = allX["propertycountylandusecode"].astype("category")
allX["fireplaceflag"] = allX["fireplaceflag"].fillna(False).astype(bool).astype("int64")
allX["taxdelinquencyflag"] = allX["taxdelinquencyflag"].replace("Y", "True")
allX["taxdelinquencyflag"] = allX["taxdelinquencyflag"].fillna(False).astype(bool).astype("int64")

In [5]:
allX_cpy = allX.copy()

In [6]:
train_X = allX_cpy.loc[train_Y.index]

In [7]:
def createDataSets():
    dataSets = {}
    # dataSets_Y = {}
    for x in train_X.columns:
        currentSeriesX = train_X[x].dropna()
        currentSeriesY = train_Y.loc[currentSeriesX.index]['logerror']
        dataSets["{0}".format(x)] = pd.DataFrame(currentSeriesX).join(pd.DataFrame(currentSeriesY))
    return dataSets

In [8]:
dataSets = createDataSets()

In [9]:
actualMean = np.mean(train_Y['logerror'])

In [10]:
regressionStats = {}

In [11]:
for key in dataSets:
    if dataSets[key][key].dtype in ['float64', 'int64']:
        slope, intercept, r_val, p_val, std_err = stats.linregress(dataSets[key][key],\
                                                                  dataSets[key]['logerror'])
        regressionStats[key] = [slope, intercept, r_val ** 2]
    else:
        pass

C:\Users\Ivan\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:106: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\Ivan\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:118: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [12]:
regressionStatsDF = pd.DataFrame.from_dict(regressionStats, orient="index")

In [13]:
regressionStatsDF[0] = regressionStatsDF[0].fillna(0)

In [14]:
for index in regressionStatsDF.index:
    if np.isnan(regressionStatsDF.loc[index][1]):
        regressionStatsDF.loc[index][1] = dataSets[index]['logerror'].mean()
    else:
        pass

In [15]:
regressionStats = regressionStatsDF.to_dict(orient="index")

In [17]:
for key_col in allX_cpy.columns:
    if allX_cpy[key_col].dtype in ['float64', 'int64']:
        try:
            unmask = np.isnan(allX_cpy[key_col])
            mask = ~np.isnan(allX_cpy[key_col])
            allX_cpy.loc[unmask, key_col] = actualMean
            allX_cpy.loc[mask, key_col] = allX_cpy.loc[mask, key_col] * regressionStats[key_col][0] + \
            regressionStats[key_col][1]
        except (KeyError, TypeError):
            allX_cpy[key_col] = actualMean
    else:
        allX_cpy[key_col] = actualMean

In [18]:
submission = pd.read_csv("sample_submission.csv", index_col='ParcelId')

C:\Users\Ivan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [19]:
allX_cpy['avg'] = allX_cpy.mean(axis=1)

In [21]:
for cols in submission.columns:
    submission[cols] = allX_cpy['avg']

In [22]:
submission.round(4)

,201610,201611,201612,201710,201711,201712
ParcelId,,,,,,
10754147,0.0111,0.0111,0.0111,0.0111,0.0111,0.0111
10759547,0.0110,0.0110,0.0110,0.0110,0.0110,0.0110
10843547,0.0239,0.0239,0.0239,0.0239,0.0239,0.0239
10859147,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125
10879947,0.0117,0.0117,0.0117,0.0117,0.0117,0.0117
10898347,0.0118,0.0118,0.0118,0.0118,0.0118,0.0118
10933547,0.0111,0.0111,0.0111,0.0111,0.0111,0.0111
10940747,0.0110,0.0110,0.0110,0.0110,0.0110,0.0110
10954547,0.0110,0.0110,0.0110,0.0110,0.0110,0.0110


In [23]:
submission.to_csv("submission8.csv")